In [14]:
print("ok")

ok


In [15]:
%pwd

'c:\\Users\\Jothsna naidu\\medical-chatbot\\research'

In [16]:
import os
os.chdir("../")

In [17]:
%pwd

'c:\\Users\\Jothsna naidu\\medical-chatbot'

In [18]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader # type: ignore
from langchain.text_splitter import RecursiveCharacterTextSplitter # type: ignore


In [19]:
#Extract Data from the pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls = PyPDFLoader)
    documents = loader.load()

    return documents

In [20]:
extracted_data  =load_pdf_file(data=r'C:\Users\Jothsna naidu\medical-chatbot\data')

In [21]:
#extracted_data

In [22]:
#chunking
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [23]:
text_chunks = text_split(extracted_data)
print("the length of text chunks",len(text_chunks))

the length of text chunks 5859


In [24]:
#text_chunks

In [25]:
from langchain_huggingface import HuggingFaceEmbeddings# type: ignore

In [26]:
#embedding model from hugging face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [27]:
embeddings=download_hugging_face_embeddings()

In [28]:
query_result = embeddings.embed_query("Hello world")
print("length",len(query_result))

length 384


In [29]:
#query_result

In [30]:
import os
print("Current Working Directory:", os.getcwd())
print("Does .env exist here?", os.path.exists(".env"))



Current Working Directory: c:\Users\Jothsna naidu\medical-chatbot
Does .env exist here? True


In [31]:
from dotenv import load_dotenv
load_dotenv(r"C:\Users\Jothsna naidu\medical-chatbot\.env")

True

In [32]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [35]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "chatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )

)


{
    "name": "chatbot",
    "metric": "cosine",
    "host": "chatbot-vtcskuj.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [36]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [37]:
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [38]:
#existing index load
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

NameError: name 'docsearch' is not defined

In [ ]:
retrieved_docs = retriever.invoke("what is Acne?")

In [ ]:
retrieved_docs

[Document(id='6245b198-43a4-43fd-8e04-1c325539b640', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\Jothsna naidu\\medical-chatbot\\data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='234578c9-151c-4a58-802b-d00e535dd238', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\Jothsna naidu\\medical-chatbot\\data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disord

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4,max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question_answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question.If you dont know the answer,say that you dont know.Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

NameError: name 'retriever' is not defined

In [ ]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************OIAA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}